In [1]:
import numpy as np

In [2]:
import sys,os
import numpy as np

from quspin.basis import spin_basis_1d, spin_basis_general
from quspin.operators import hamiltonian,quantum_operator

os.environ['KMP_DUPLICATE_LIB_OK']='True' # uncomment this line if omp error occurs on OSX for python 3
os.environ['OMP_NUM_THREADS']='1' # set number of OpenMP threads to run in parallel
os.environ['MKL_NUM_THREADS']='12' # set number of MKL threads to run in parallel

In [3]:
(4 ** (16)) * 64

274877906944

In [3]:
Lx,Ly = 4, 4
N_2d = Lx*Ly
Jz, Jx, Jy, h = [1, -0.3, 0.5, 1]
s = np.arange(N_2d)
x = s%Lx 
y = s//Lx 
T_x = (x+1)%Lx + Lx*y 
T_y = x +Lx*((y+1)%Ly) 

In [4]:
J_zz = [[1,i,T_x[i]] for i in range(N_2d)]+[[1,i,T_y[i]] for i in range(N_2d)] 
h_list = [[-1.0,i] for i in range(N_2d)]
ops_dict = dict(Jz=[["zz", J_zz]], # J for z
                Jx=[["xx",J_zz]],
                Jy=[["yy",J_zz]], # J for x and y
                h=[["z",h_list]])
M_list = [[1.0/N_2d,i] for i in range(N_2d)]

basis_2d = spin_basis_general(N_2d ,pauli=False)
_H = quantum_operator(ops_dict,basis=basis_2d, dtype=np.float64, check_symm=False)
M = hamiltonian([["z",M_list]],[],basis=basis_2d, dtype=np.float64)
params_dict=dict(Jz=Jz, Jx=Jx,Jy=Jy,h=h)

Hermiticity check passed!
Hermiticity check passed!


/tmp/ipykernel_14324/560145047.py:11: UserWarning: Test for symmetries not implemented for <class 'quspin.basis.basis_general.spin.spin_basis_general'>, to turn off this warning set check_symm=False in hamiltonian
  M = hamiltonian([["z",M_list]],[],basis=basis_2d, dtype=np.float64)


In [5]:
import cupy as cp

In [6]:
Hs = _H.tohamiltonian(params_dict)
H = Hs.toarray()
H_gpu = cp.asarray(H)

OutOfMemoryError: Out of memory allocating 34,359,738,368 bytes (allocated so far: 0 bytes).

In [50]:
E, V = cp.linalg.eigh(H_gpu)

In [51]:
M_np = M.toarray()
M_gpu = cp.asarray(M_np)

In [52]:
T = 0.4
beta = 1/T
Z = cp.exp(-beta*E).sum()
M_mean = cp.einsum("n,in,jn,ij->", cp.exp(-beta*E), V, V, M_gpu) / Z
M2_mean = cp.einsum("n,in,jn,ij->", cp.exp(-beta*E), V, V, M_gpu@M_gpu) / Z
E_mean = (E*cp.exp(-beta*E)).sum() / Z
E2_mean = ((E*E)*cp.exp(-beta*E)).sum() / Z

In [53]:
print(params_dict)
print(f"T               = {T}")
print(f"E               = {E_mean / N_2d}")
print(f"C               = {(E2_mean - E_mean**2)*(beta**2)/N_2d}")
print(f"M               = {M_mean}")
print(f"M^2             = {M2_mean}")

{'Jz': 1, 'Jx': -0.3, 'Jy': 0.5, 'h': 1}
T               = 0.4
E               = -0.31290500262456744
C               = 0.20476937630462452
M               = 0.11860093087143614
M^2             = 0.02034135779893476


In [54]:
print(f"Chi             = {(0.11860093087143614 - 0.11721013936308525)/0.01}")

Chi             = 0.13907915083508915
